<a href="https://colab.research.google.com/github/AnayPanja/CNN-Model-using-Keras-Tuner/blob/main/CNN_Model_using_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 3.5 MB/s 


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [4]:
print(tf.__version__)

2.7.0


In [5]:
 # here I use fashion mnist dataset from tf library

 data = keras.datasets.fashion_mnist

In [6]:
(train_images,train_labels),(test_images,test_labels) = data.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [7]:
# scale down the images
train_images = train_images/255.0
test_images = test_images/255.0

In [8]:
train_images[1].shape

(28, 28)

In [9]:
len(train_images)


60000

In [10]:
len(test_images)

10000

In [11]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [12]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [14]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='/content/drive/MyDrive/Colab Notebooks/output/CNN Model using Keras Tuner')

INFO:tensorflow:Reloading Oracle from existing project /content/drive/MyDrive/Colab Notebooks/output/CNN Model using Keras Tuner/untitled_project/oracle.json


In [15]:
tuner_search.search(train_images,train_labels,epochs=2,validation_split=0.1)

Trial 6 Complete [00h 00m 31s]
val_accuracy: 0.8981666564941406

Best val_accuracy So Far: 0.903166651725769
Total elapsed time: 00h 03m 40s
INFO:tensorflow:Oracle triggered exit


In [16]:
model=tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 96)        960       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        27680     
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 96)                1769568   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 1,799,178
Trainable params: 1,799,178
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(train_images,train_labels,validation_split=0.1,epochs=10,initial_epoch=2)

Epoch 3/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.2389 - accuracy: 0.9129 - val_loss: 0.2649 - val_accuracy: 0.9032
Epoch 4/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.1761 - accuracy: 0.9347 - val_loss: 0.2705 - val_accuracy: 0.9037
Epoch 5/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.1254 - accuracy: 0.9541 - val_loss: 0.2700 - val_accuracy: 0.9150
Epoch 6/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0866 - accuracy: 0.9679 - val_loss: 0.3118 - val_accuracy: 0.9090
Epoch 7/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0597 - accuracy: 0.9775 - val_loss: 0.3725 - val_accuracy: 0.9158
Epoch 8/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0439 - accuracy: 0.9837 - val_loss: 0.4279 - val_accuracy: 0.9147
Epoch 9/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0326 - accuracy: 0.9886 - val_loss: 0.4647 - va